In [1]:
import pandas as pd
from sunpy.coordinates import HeliographicStonyhurst, Helioprojective
import astropy.units as u
from astropy.coordinates import SkyCoord
from sunpy.net import Fido
from sunpy.net import attrs as a
from sunpy.map import Map
from sunpy.coordinates import frames, get_body_heliographic_stonyhurst, get_horizons_coord, transform_with_sun_center
import matplotlib.pyplot as plt

In [2]:
df1 = pd.read_csv('flare_location_output4_part1.csv')
df2 = pd.read_csv('flare_location_output4_part2.csv')

In [3]:
df1.head()

,flare ID,request ID,X flare loc,Y flare loc,FWDFIT X SIGMA,FWDFIT Y SIGMA,MAX X COORD BP,MAX Y COORD BP
0,2211090050,2211090305,-15.342900,181.460000,1.029180,1.327330,-16.415800,180.574000
1,2211090720,2211091049,359.304596,523.128906,0.532592,0.566841,359.168732,522.427246
2,2211090908,2211096407,377.918396,529.199951,1.988731,1.876073,374.929199,529.791260
3,2211090958,2211091409,824.236572,503.752686,2.422804,2.067823,822.638184,496.840912
4,2211091013,2211091673,407.905670,450.271332,1.130224,1.113955,407.163910,447.880310


In [4]:
len(df1), len(df2)

(185, 185)

In [5]:
df2.head()

,flare ID,STIX MAX X BP COORD,STIX MAX Y BP COORD,time shift
0,2211090050,97.614000,35.544700,198.206746
1,2211090720,-202.187622,-1784.592041,196.524467
2,2211090908,-193.186462,-1797.633179,196.084910
3,2211090958,-218.136597,-2244.614746,195.855872
4,2211091013,-274.121002,-1829.591797,195.800365


In [6]:
new_df = df2.join(df1.set_index('flare ID'), on='flare ID', how='right')

In [7]:
len(new_df)

185

In [8]:
full_fl_paths = pd.read_csv('full_flarelist_with_paths.csv')

In [9]:
full_fl_paths.head()

,flare_id,att_in,GOES_flux,start_UTC,peak_UTC,end_UTC,4-10 keV,10-15 keV,number_available_files,available_file_request_IDs,solo_lat,solo_lon,solo_AU_distance,15-25 keV,25-50 keV,50-84 keV,GOES_class,Request IDs,aux_paths,pixel_paths
0,2211090050,False,6.073233e-07,2022-11-09T00:44:44.012,2022-11-09 00:50:20.013,2022-11-09T00:52:56.013,1215,271,1,[2211090305],9.180063,-24.156920,0.590873,135,799,463,B6.1,2211090305,./aux_data/solo_L2_stix-aux-ephemeris_20221109...,./pixel_data/solo_L1_stix-sci-xray-cpd_2022110...
1,2211090720,False,9.645386e-07,2022-11-09T07:18:12.052,2022-11-09 07:20:44.052,2022-11-09T07:28:40.053,2431,247,1,[2211091049],9.176525,-23.910781,0.594137,115,799,463,B9.6,2211091049,./aux_data/solo_L2_stix-aux-ephemeris_20221109...,./pixel_data/solo_L1_stix-sci-xray-cpd_2022110...
2,2211090908,False,6.582638e-07,2022-11-09T08:58:40.062,2022-11-09 09:08:44.063,2022-11-09T09:14:56.064,607,135,1,[2211096407],9.175424,-23.843695,0.595037,107,863,463,B6.6,2211096407,./aux_data/solo_L2_stix-aux-ephemeris_20221109...,./pixel_data/solo_L1_stix-sci-xray-cpd_2022110...
3,2211090958,False,6.191904e-07,2022-11-09T09:55:28.068,2022-11-09 09:58:44.069,2022-11-09T10:06:04.069,607,75,1,[2211091409],9.174897,-23.812783,0.595453,75,799,463,B6.2,2211091409,./aux_data/solo_L2_stix-aux-ephemeris_20221109...,./pixel_data/solo_L1_stix-sci-xray-cpd_2022110...
4,2211091013,False,6.162408e-07,2022-11-09T10:11:28.070,2022-11-09 10:13:24.070,2022-11-09T10:16:24.070,735,115,1,[2211091673],9.174741,-23.803733,0.595575,75,799,463,B6.2,2211091673,./aux_data/solo_L2_stix-aux-ephemeris_20221109...,./pixel_data/solo_L1_stix-sci-xray-cpd_2022110...


In [10]:
final_df = pd.merge(full_fl_paths, new_df, right_on ='flare ID', left_on = 'flare_id')

In [11]:
len(final_df)

185

In [12]:
final_df.head()

,flare_id,att_in,GOES_flux,start_UTC,peak_UTC,end_UTC,4-10 keV,10-15 keV,number_available_files,available_file_request_IDs,...,STIX MAX X BP COORD,STIX MAX Y BP COORD,time shift,request ID,X flare loc,Y flare loc,FWDFIT X SIGMA,FWDFIT Y SIGMA,MAX X COORD BP,MAX Y COORD BP
0,2211090050,False,6.073233e-07,2022-11-09T00:44:44.012,2022-11-09 00:50:20.013,2022-11-09T00:52:56.013,1215,271,1,[2211090305],...,97.614000,35.544700,198.206746,2211090305,-15.342900,181.460000,1.029180,1.327330,-16.415800,180.574000
1,2211090720,False,9.645386e-07,2022-11-09T07:18:12.052,2022-11-09 07:20:44.052,2022-11-09T07:28:40.053,2431,247,1,[2211091049],...,-202.187622,-1784.592041,196.524467,2211091049,359.304596,523.128906,0.532592,0.566841,359.168732,522.427246
2,2211090908,False,6.582638e-07,2022-11-09T08:58:40.062,2022-11-09 09:08:44.063,2022-11-09T09:14:56.064,607,135,1,[2211096407],...,-193.186462,-1797.633179,196.084910,2211096407,377.918396,529.199951,1.988731,1.876073,374.929199,529.791260
3,2211090958,False,6.191904e-07,2022-11-09T09:55:28.068,2022-11-09 09:58:44.069,2022-11-09T10:06:04.069,607,75,1,[2211091409],...,-218.136597,-2244.614746,195.855872,2211091409,824.236572,503.752686,2.422804,2.067823,822.638184,496.840912
4,2211091013,False,6.162408e-07,2022-11-09T10:11:28.070,2022-11-09 10:13:24.070,2022-11-09T10:16:24.070,735,115,1,[2211091673],...,-274.121002,-1829.591797,195.800365,2211091673,407.905670,450.271332,1.130224,1.113955,407.163910,447.880310


In [13]:
final_df.keys()

Index(['flare_id', 'att_in', 'GOES_flux', 'start_UTC', 'peak_UTC', 'end_UTC',
       '4-10 keV', '10-15 keV', 'number_available_files',
       'available_file_request_IDs', 'solo_lat', 'solo_lon',
       'solo_AU_distance', '15-25 keV', '25-50 keV', '50-84 keV', 'GOES_class',
       'Request IDs', 'aux_paths', 'pixel_paths', 'flare ID',
       'STIX MAX X BP COORD', 'STIX MAX Y BP COORD', 'time shift',
       'request ID', 'X flare loc', 'Y flare loc', 'FWDFIT X SIGMA',
       'FWDFIT Y SIGMA', 'MAX X COORD BP', 'MAX Y COORD BP'],
      dtype='object')

In [14]:
earth_x_helioprojective = []
earth_y_helioprojective = []

earth_x_heliographic = []
earth_y_heliographic = []

for i in range(len(final_df)):
    x = final_df.iloc[i]['X flare loc']
    y = final_df.iloc[i]['Y flare loc']
    obstime = final_df.iloc[i]['peak_UTC'] 
    solo_pos = get_horizons_coord("solar orbiter",obstime)
    earth_coord_helioprojective = SkyCoord(x*u.arcsec, y*u.arcsec, frame=Helioprojective, observer=solo_pos).transform_to(Helioprojective(observer="earth", obstime=obstime))
    earth_coord_heliocentric = SkyCoord(x*u.arcsec, y*u.arcsec, frame=Helioprojective, observer=solo_pos).transform_to(HeliographicStonyhurst(obstime=obstime))
    
    earth_x_heliographic.append(earth_coord_heliocentric.lon.value)
    earth_y_heliographic.append(earth_coord_heliocentric.lat.value)
    
    earth_x_helioprojective.append(earth_coord_helioprojective.Tx.value)
    earth_y_helioprojective.append(earth_coord_helioprojective.Ty.value)
    

INFO: Obtained JPL HORIZONS location for Solar Orbiter (spacecraft) (-144 [sunpy.coordinates.ephemeris]
INFO: Obtained JPL HORIZONS location for Solar Orbiter (spacecraft) (-144 [sunpy.coordinates.ephemeris]
INFO: Obtained JPL HORIZONS location for Solar Orbiter (spacecraft) (-144 [sunpy.coordinates.ephemeris]
INFO: Obtained JPL HORIZONS location for Solar Orbiter (spacecraft) (-144 [sunpy.coordinates.ephemeris]
INFO: Obtained JPL HORIZONS location for Solar Orbiter (spacecraft) (-144 [sunpy.coordinates.ephemeris]
INFO: Obtained JPL HORIZONS location for Solar Orbiter (spacecraft) (-144 [sunpy.coordinates.ephemeris]
INFO: Obtained JPL HORIZONS location for Solar Orbiter (spacecraft) (-144 [sunpy.coordinates.ephemeris]
INFO: Obtained JPL HORIZONS location for Solar Orbiter (spacecraft) (-144 [sunpy.coordinates.ephemeris]
INFO: Obtained JPL HORIZONS location for Solar Orbiter (spacecraft) (-144 [sunpy.coordinates.ephemeris]
INFO: Obtained JPL HORIZONS location for Solar Orbiter (spacecra

INFO: Obtained JPL HORIZONS location for Solar Orbiter (spacecraft) (-144 [sunpy.coordinates.ephemeris]
INFO: Obtained JPL HORIZONS location for Solar Orbiter (spacecraft) (-144 [sunpy.coordinates.ephemeris]
INFO: Obtained JPL HORIZONS location for Solar Orbiter (spacecraft) (-144 [sunpy.coordinates.ephemeris]
INFO: Obtained JPL HORIZONS location for Solar Orbiter (spacecraft) (-144 [sunpy.coordinates.ephemeris]
INFO: Obtained JPL HORIZONS location for Solar Orbiter (spacecraft) (-144 [sunpy.coordinates.ephemeris]
INFO: Obtained JPL HORIZONS location for Solar Orbiter (spacecraft) (-144 [sunpy.coordinates.ephemeris]
INFO: Obtained JPL HORIZONS location for Solar Orbiter (spacecraft) (-144 [sunpy.coordinates.ephemeris]
INFO: Obtained JPL HORIZONS location for Solar Orbiter (spacecraft) (-144 [sunpy.coordinates.ephemeris]
INFO: Obtained JPL HORIZONS location for Solar Orbiter (spacecraft) (-144 [sunpy.coordinates.ephemeris]
INFO: Obtained JPL HORIZONS location for Solar Orbiter (spacecra

INFO: Obtained JPL HORIZONS location for Solar Orbiter (spacecraft) (-144 [sunpy.coordinates.ephemeris]
INFO: Obtained JPL HORIZONS location for Solar Orbiter (spacecraft) (-144 [sunpy.coordinates.ephemeris]
INFO: Obtained JPL HORIZONS location for Solar Orbiter (spacecraft) (-144 [sunpy.coordinates.ephemeris]
INFO: Obtained JPL HORIZONS location for Solar Orbiter (spacecraft) (-144 [sunpy.coordinates.ephemeris]
INFO: Obtained JPL HORIZONS location for Solar Orbiter (spacecraft) (-144 [sunpy.coordinates.ephemeris]
INFO: Obtained JPL HORIZONS location for Solar Orbiter (spacecraft) (-144 [sunpy.coordinates.ephemeris]
INFO: Obtained JPL HORIZONS location for Solar Orbiter (spacecraft) (-144 [sunpy.coordinates.ephemeris]
INFO: Obtained JPL HORIZONS location for Solar Orbiter (spacecraft) (-144 [sunpy.coordinates.ephemeris]
INFO: Obtained JPL HORIZONS location for Solar Orbiter (spacecraft) (-144 [sunpy.coordinates.ephemeris]
INFO: Obtained JPL HORIZONS location for Solar Orbiter (spacecra

INFO: Obtained JPL HORIZONS location for Solar Orbiter (spacecraft) (-144 [sunpy.coordinates.ephemeris]
INFO: Obtained JPL HORIZONS location for Solar Orbiter (spacecraft) (-144 [sunpy.coordinates.ephemeris]
INFO: Obtained JPL HORIZONS location for Solar Orbiter (spacecraft) (-144 [sunpy.coordinates.ephemeris]


In [15]:
final_df["Earth X Helioprojective"] = earth_x_helioprojective 
final_df['Earth Y Helioprojective'] = earth_y_helioprojective

final_df["Lon Heliographic"] = earth_x_heliographic
final_df['Lat Heliographic'] = earth_y_heliographic

In [16]:
final_df.keys()

Index(['flare_id', 'att_in', 'GOES_flux', 'start_UTC', 'peak_UTC', 'end_UTC',
       '4-10 keV', '10-15 keV', 'number_available_files',
       'available_file_request_IDs', 'solo_lat', 'solo_lon',
       'solo_AU_distance', '15-25 keV', '25-50 keV', '50-84 keV', 'GOES_class',
       'Request IDs', 'aux_paths', 'pixel_paths', 'flare ID',
       'STIX MAX X BP COORD', 'STIX MAX Y BP COORD', 'time shift',
       'request ID', 'X flare loc', 'Y flare loc', 'FWDFIT X SIGMA',
       'FWDFIT Y SIGMA', 'MAX X COORD BP', 'MAX Y COORD BP',
       'Earth X Helioprojective', 'Earth Y Helioprojective',
       'Lon Heliographic', 'Lat Heliographic'],
      dtype='object')

In [17]:
filtered = final_df[final_df['4-10 keV']>5*10**2]

In [19]:
len(final_df)

185

In [20]:
len(filtered)

185

In [24]:
final_df.to_csv('091122-151122_final_full_flare_list.csv')

In [25]:
filtered.to_csv('091122-151122_final_filtered_flare_list.csv')

In [16]:
len(final_df)

47

In [17]:
len(filtered)

41

In [22]:
filtered

,flare_id,att_in,GOES_flux,start_UTC,peak_UTC,end_UTC,4-10 keV,10-15 keV,number_available_files,available_file_request_IDs,...,X flare loc,Y flare loc,FWDFIT X SIGMA,FWDFIT Y SIGMA,MAX X COORD BP,MAX Y COORD BP,Earth X Helioprojective,Earth Y Helioprojective,Lon Heliographic,Lat Heliographic
0,2211090050,False,6.073233e-07,2022-11-09T00:44:44.012,2022-11-09 00:50:20.013,2022-11-09T00:52:56.013,1215,271,1,[2211090305],...,-15.342900,181.460000,1.029180,1.327330,-16.415800,180.574000,-382.780474,187.116567,-23.974729,14.351437
1,2211090720,False,9.645386e-07,2022-11-09T07:18:12.052,2022-11-09 07:20:44.052,2022-11-09T07:28:40.053,2431,247,1,[2211091049],...,359.304596,523.128906,0.532592,0.566841,359.168732,522.427246,-133.265261,381.172357,-8.812965,26.565168
2,2211090908,False,6.582638e-07,2022-11-09T08:58:40.062,2022-11-09 09:08:44.063,2022-11-09T09:14:56.064,607,135,1,[2211096407],...,377.918396,529.199951,1.988731,1.876073,374.929199,529.791260,-119.781288,384.758337,-7.931052,26.793971
3,2211090958,False,6.191904e-07,2022-11-09T09:55:28.068,2022-11-09 09:58:44.069,2022-11-09T10:06:04.069,607,75,1,[2211091409],...,824.236572,503.752686,2.422804,2.067823,822.638184,496.840912,170.810024,354.431377,11.158014,24.825639
4,2211091013,False,6.162408e-07,2022-11-09T10:11:28.070,2022-11-09 10:13:24.070,2022-11-09T10:16:24.070,735,115,1,[2211091673],...,407.905670,450.271332,1.130224,1.113955,407.163910,447.880310,-109.160179,338.202025,-7.048012,23.838437
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
180,2211142151,False,4.388000e-06,2022-11-14T21:29:48.099,2022-11-14 21:51:20.101,2022-11-14T22:08:04.103,29695,2943,2,"[2211145517, 2211140731]",...,1331.421509,519.345093,0.182756,0.218209,1326.779419,515.969788,797.264278,346.265471,62.435442,22.238771
181,2211142250,False,2.930119e-06,2022-11-14T22:19:44.104,2022-11-14 22:50:36.107,2022-11-14T23:08:28.109,6399,463,2,"[2211147867, 2211143309]",...,1354.642090,495.102631,0.365246,0.622751,1347.982910,493.523804,833.884019,324.445845,66.577668,20.682335
182,2211142310,False,2.244843e-06,2022-11-14T23:09:56.109,2022-11-14 23:10:52.110,2022-11-14T23:11:48.110,543,67,1,[2211147867],...,1355.483154,543.438477,12.000789,12.080571,1347.672485,537.596130,NaN,NaN,NaN,NaN
183,2211142324,False,1.959793e-06,2022-11-14T23:23:56.111,2022-11-14 23:24:48.111,2022-11-14T23:25:48.111,735,91,1,[2211141248],...,1331.372559,518.049194,1.377404,1.230874,1332.732788,515.421570,799.614897,345.503128,62.710655,22.174879


# Transformation sanity check for known event


In [2]:
solo_pos_march22 = get_horizons_coord("solar orbiter", "2022-03-30")


INFO: Obtained JPL HORIZONS location for Solar Orbiter (spacecraft) (-144 [sunpy.coordinates.ephemeris]


In [3]:
solo_pos_march22

<SkyCoord (HeliographicStonyhurst: obstime=2022-03-30T00:00:00.000, rsun=695700.0 km): (lon, lat, radius) in (deg, deg, AU)
    (91.74058012, 1.30847464, 0.33146424)>

In [4]:
earth_coord_march22 = SkyCoord(-2485*u.arcsec, 600*u.arcsec, frame=Helioprojective, observer=solo_pos_march22).transform_to(Helioprojective(observer="earth", obstime='2022-03-30'))


In [5]:
earth_coord_march22

<SkyCoord (Helioprojective: obstime=2022-03-30T00:00:00.000, rsun=695700.0 km, observer=<HeliographicStonyhurst Coordinate for 'earth'>): (Tx, Ty, distance) in (arcsec, arcsec, AU)
    (482.59909625, 301.44357158, 0.99477816)>

In [16]:
c = SkyCoord(-2485*u.arcsec, 600*u.arcsec, frame=Helioprojective, observer=solo_pos_march22).transform_to(HeliographicStonyhurst(obstime='2022-03-30'))



In [22]:
c

<SkyCoord (HeliographicStonyhurst: obstime=2022-03-30T00:00:00.000, rsun=695700.0 km): (lon, lat, radius) in (deg, deg, AU)
    (30.84273515, 12.52413733, 0.00465047)>

In [24]:
c.lon, c.lat, c.radius

(<Longitude 30.84273515 deg>,
 <Latitude 12.52413733 deg>,
 <Distance 0.00465047 AU>)